# LOAD Modules

In [1]:
%matplotlib inline
#%matplotlib widget
%load_ext autoreload
%autoreload 2


import numpy as np
from visualisations import eeg_sample_plot, events_distribution_plot
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import DataLoader
from datasets import ChannelsVoltageDataset
from neural_nets.nn_models_getter import get_nn_model
from optimizers import get_optimizer
from utils_train import fit, test, plot_all_metrics, plot_val_metrics
from configs import configs_ozhan, configs_tim, configs_joaquin
from data_loader_creation import get_dataloader_objects
from classification_results import results_storer



# Select the configuration

In [ ]:
""" USER: SELECT THE CONFIGURATION YOU NEED """
#myList = configs_tim.list_of_configs
#myList = configs_joaquin.list_of_configs
myList = configs_ozhan.list_of_configs

# Train

In [ ]:
training_curves = {}

for my_cfg in myList:
    
    curve_name = my_cfg.nn_list[my_cfg.nn_selection_idx] + "_" + my_cfg.optimizer_list[my_cfg.optimizer_selection_idx]
    print(curve_name,"\n-------------------------\n", flush=True)
    
    """ PREPARE DATALOADERS """
    train_dl, val_dl, test_dl, input_dimension_, output_dimension_ = get_dataloader_objects(my_cfg)


    """CLASSIFICATION"""
    # Get the model
    model_untrained = get_nn_model(my_cfg.nn_list[my_cfg.nn_selection_idx], input_dimension=input_dimension_,
                                   output_dimension=output_dimension_, dropout=my_cfg.dropout_perc)

    # Get the optimizer
    optimizer = get_optimizer(my_cfg.optimizer_list[my_cfg.optimizer_selection_idx], my_cfg.learning_rate,
                              model_untrained.parameters(), my_cfg.momentum, my_cfg.weight_decay)

    # Train and show validation loss
    train_losses, train_accuracies, val_losses, val_accuracies, model_trained, time_spent_for_training_s =\
        fit(train_dl, val_dl, model_untrained, optimizer, my_cfg.loss_fn, my_cfg.num_of_epochs, \
            scheduler=my_cfg.scheduler, apply_early_stopping = my_cfg.use_early_stopping, \
            estop_patience=my_cfg.es_patience, curve_name = curve_name)
    
    
    curves = train_losses, train_accuracies, val_losses, val_accuracies, model_trained, time_spent_for_training_s
    
    
    training_curves[curve_name] = curves
    
    # Test the net
    print('\nPerformance on the test set:')
    test_loss, test_accuracy = test(model_trained, test_dl, my_cfg.loss_fn, print_loss=True)
    print('->test_loss: {:.4f}, test_accuracy: {:.4f}%'.format(test_loss, test_accuracy))


    # Store the results
    results_storer.store_results_for_plot(my_cfg,test_loss, test_accuracy, train_losses,
                                 train_accuracies, time_spent_for_training_s, val_losses, val_accuracies)

    results_storer.store_results(my_cfg, model_trained, optimizer, test_loss, test_accuracy, train_losses,
                                 train_accuracies, time_spent_for_training_s, val_losses, val_accuracies, test_dl)


plot_all_metrics(training_curves)

ConvNetOzhan_Adam 
-------------------------

We are not on the cluster...

Data is being loaded using MNE...


100%|██████████| 6/6 [00:31<00:00,  5.82s/it]


5418 matching events found
No baseline correction applied
...data loading with MNE was finished. 

Data is being augmented with gaussian noise...


100%|██████████| 4388/4388 [00:24<00:00, 179.14it/s]


...augmentation with gaussian noise is finished. 

cuda will be used for training of this model.
You have 2 GPUs for training!
Learning Rate:  0.001 

-> Epoch 1/50: train_loss: 1.8900, train_accuracy: 27.4499%, val_loss: 1.7768, val_accuracy: 31.1475%
-> Epoch 2/50: train_loss: 1.7244, train_accuracy: 34.7767%, val_loss: 1.9186, val_accuracy: 27.0492%
-> Epoch 3/50: train_loss: 1.6709, train_accuracy: 36.7935%, val_loss: 1.8434, val_accuracy: 29.9180%
-> Epoch 4/50: train_loss: 1.6095, train_accuracy: 40.2746%, val_loss: 1.8662, val_accuracy: 31.9672%
-> Epoch 5/50: train_loss: 1.5470, train_accuracy: 43.3968%, val_loss: 1.8541, val_accuracy: 28.4836%
-> Epoch 6/50: train_loss: 1.4692, train_accuracy: 46.3366%, val_loss: 1.6761, val_accuracy: 35.0410%
-> Epoch 7/50: train_loss: 1.3697, train_accuracy: 50.3247%, val_loss: 1.7324, val_accuracy: 36.6803%
-> Epoch 8/50: train_loss: 1.2416, train_accuracy: 55.6233%, val_loss: 1.9161, val_accuracy: 28.8934%
-> Epoch 9/50: train_loss: 1.1021